Q. [마케팅] 자동차 시장 세분화

    자동차 회사는 새로운 전략을 수립하기 위해 4개의 시장으로 세분화했습니다.

    기존 고객 분류 자료를 바탕으로 신규 고객이 어떤 분류에 속할지 예측해주세요!

    예측할 값(y): "Segmentation" (1,2,3,4)

    평가: Macro f1-score

    data: train.csv, test.csv

    제출 형식:

ID,Segmentation
458989,1
458994,2
459000,3
459003,4

답안 제출 참고

    아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
    pd.DataFrame({'ID': test.ID, 'Segmentation': pred}).to_csv('003000000.csv', index=False)

노트북 구분

    basic: 수치형 데이터만 활용 -> 학습 및 test데이터 예측
    intermediate: 범주형 데이터도 활용 -> 학습 및 test데이터 예측
    advanced: 학습 및 교차 검증(모델 평가) -> 하이퍼파라미터 튜닝 -> test데이터 예측

학습을 위한 채점

    최종 파일을 "수험번호.csv"가 아닌 "submission.csv" 작성 후 오른쪽 메뉴 아래 "submit" 버튼 클릭 -> 리더보드에 점수 및 등수 확인 가능함
    pd.DataFrame({'ID': test.ID, 'Segmentation': pred}).to_csv('submission.csv', index=False)



In [138]:
import pandas as pd
import numpy as np

In [140]:
train =pd.read_csv('./data/4th-t2/train.csv')
test=pd.read_csv('data/4th-t2/test.csv')

In [142]:
train.shape, test.shape

((6665, 11), (2154, 10))

In [144]:
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,4
1,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,2
2,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,2
3,461319,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6,3
4,460156,Male,No,32,Yes,Healthcare,1.0,Low,3.0,Cat_6,3


In [146]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6665 non-null   int64  
 1   Gender           6665 non-null   object 
 2   Ever_Married     6665 non-null   object 
 3   Age              6665 non-null   int64  
 4   Graduated        6665 non-null   object 
 5   Profession       6665 non-null   object 
 6   Work_Experience  6665 non-null   float64
 7   Spending_Score   6665 non-null   object 
 8   Family_Size      6665 non-null   float64
 9   Var_1            6665 non-null   object 
 10  Segmentation     6665 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 572.9+ KB


In [148]:
train.describe()

,ID,Age,Work_Experience,Family_Size,Segmentation
count,6665.00000,6665.000000,6665.000000,6665.000000,6665.000000
mean,463519.84096,43.536084,2.629107,2.841110,2.542836
std,2566.43174,16.524054,3.405365,1.524743,1.122723
min,458982.00000,18.000000,0.000000,1.000000,1.000000
25%,461349.00000,31.000000,0.000000,2.000000,2.000000
50%,463575.00000,41.000000,1.000000,2.000000,3.000000
75%,465741.00000,53.000000,4.000000,4.000000,4.000000
max,467974.00000,89.000000,14.000000,9.000000,4.000000


# basic 단계 
- 수치형 데이터만 활용해 제출

In [151]:
# test 샘플 확인 
test.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,459000,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6
3,459003,Male,Yes,47,Yes,Doctor,0.0,High,5.0,Cat_4
4,459005,Male,Yes,61,Yes,Doctor,5.0,Low,3.0,Cat_6


In [154]:
# target 확인
train['Segmentation'].value_counts()

Segmentation
4    1757
3    1720
1    1616
2    1572
Name: count, dtype: int64

In [160]:
# 결측치 확인(train)
# train.isnull().sum()

In [158]:
# 결측치 확인(test)
# test.isnull().sum()

In [80]:
# type 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6665 non-null   int64  
 1   Gender           6665 non-null   object 
 2   Ever_Married     6665 non-null   object 
 3   Age              6665 non-null   int64  
 4   Graduated        6665 non-null   object 
 5   Profession       6665 non-null   object 
 6   Work_Experience  6665 non-null   float64
 7   Spending_Score   6665 non-null   object 
 8   Family_Size      6665 non-null   float64
 9   Var_1            6665 non-null   object 
 10  Segmentation     6665 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 572.9+ KB


In [162]:
# target(y, label) 값 복사
target = train.pop('Segmentation')
target

0       4
1       2
2       2
3       3
4       3
       ..
6660    2
6661    4
6662    4
6663    2
6664    2
Name: Segmentation, Length: 6665, dtype: int64

In [164]:
train = train.drop("ID", axis=1)
train.head(1)

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4


In [166]:
# test데이터 ID 복사
test_ID = test.pop('ID')

In [168]:
# 수치형 컬럼(train)
# ['ID', 'Age', 'Work_Experience', 'Family_Size', 'Segmentation']
num_cols = ['Age', 'Work_Experience', 'Family_Size']
train = train[num_cols]
train.head(2)

,Age,Work_Experience,Family_Size
0,22,1.0,4.0
1,67,1.0,1.0


In [170]:
# 수치형 컬럼(test)
test = test[num_cols]
test.head(2)

,Age,Work_Experience,Family_Size
0,36,0.0,1.0
1,37,8.0,4.0


## model 학습 및 예측

In [173]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(train, target)
pred = rf.predict(test)
pred

array([2, 3, 3, ..., 4, 3, 1], dtype=int64)

In [175]:
submit=pd.DataFrame({
    'ID':test_ID,
    'segmentation': pred
})
submit

,ID,segmentation
0,458989,2
1,458994,3
2,459000,3
3,459003,3
4,459005,2
...,...,...
2149,467950,1
2150,467954,4
2151,467958,4
2152,467961,3


In [177]:
submit.to_csv("submission.csv", index=False)
# Score: 0.30477

# intermediate 단계

- 목표: 범주형(카테고리)데이터 활용하기

In [179]:
train =pd.read_csv('./data/4th-t2/train.csv')
test=pd.read_csv('data/4th-t2/test.csv')

In [181]:
train.head(2)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,4
1,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,2


In [183]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6665 non-null   int64  
 1   Gender           6665 non-null   object 
 2   Ever_Married     6665 non-null   object 
 3   Age              6665 non-null   int64  
 4   Graduated        6665 non-null   object 
 5   Profession       6665 non-null   object 
 6   Work_Experience  6665 non-null   float64
 7   Spending_Score   6665 non-null   object 
 8   Family_Size      6665 non-null   float64
 9   Var_1            6665 non-null   object 
 10  Segmentation     6665 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 572.9+ KB


In [185]:
train.describe(include="O") # 범주형(object 타입) 변수들에 대한 요약 통계 

,Gender,Ever_Married,Graduated,Profession,Spending_Score,Var_1
count,6665,6665,6665,6665,6665,6665
unique,2,2,2,9,3,7
top,Male,Yes,Yes,Artist,Low,Cat_6
freq,3677,3944,4249,2192,3999,4476


## 전처리

In [188]:
train= pd.get_dummies(train)
test=pd.get_dummies(test)

In [190]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        6665 non-null   int64  
 1   Age                       6665 non-null   int64  
 2   Work_Experience           6665 non-null   float64
 3   Family_Size               6665 non-null   float64
 4   Segmentation              6665 non-null   int64  
 5   Gender_Female             6665 non-null   bool   
 6   Gender_Male               6665 non-null   bool   
 7   Ever_Married_No           6665 non-null   bool   
 8   Ever_Married_Yes          6665 non-null   bool   
 9   Graduated_No              6665 non-null   bool   
 10  Graduated_Yes             6665 non-null   bool   
 11  Profession_Artist         6665 non-null   bool   
 12  Profession_Doctor         6665 non-null   bool   
 13  Profession_Engineer       6665 non-null   bool   
 14  Professi

In [192]:
target= train.pop('Segmentation')
target

0       4
1       2
2       2
3       3
4       3
       ..
6660    2
6661    4
6662    4
6663    2
6664    2
Name: Segmentation, Length: 6665, dtype: int64

In [194]:
train= train.drop('ID', axis=1)
train.head(1)

,Age,Work_Experience,Family_Size,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Graduated_No,Graduated_Yes,Profession_Artist,...,Spending_Score_Average,Spending_Score_High,Spending_Score_Low,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
0,22,1.0,4.0,False,True,True,False,True,False,False,...,False,False,True,False,False,False,True,False,False,False


In [196]:
test_ID=test.pop('ID')
test_ID

0       458989
1       458994
2       459000
3       459003
4       459005
         ...  
2149    467950
2150    467954
2151    467958
2152    467961
2153    467968
Name: ID, Length: 2154, dtype: int64

In [204]:
# 모델 선택 및 학습
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier()
rf.fit(train,target)
pred=rf.predict(test)
pred

array([2, 3, 3, ..., 2, 3, 4], dtype=int64)

In [206]:
submit= pd.DataFrame({
    'ID':test_ID,
    'Segmentation':pred})
submit

,ID,Segmentation
0,458989,2
1,458994,3
2,459000,3
3,459003,3
4,459005,1
...,...,...
2149,467950,1
2150,467954,4
2151,467958,2
2152,467961,3


In [208]:
submit.to_csv('submission.csv',index=False)

# Advanced 단계

- 목표: 교차검증 및 평가 후 제출하기

In [213]:
train =pd.read_csv('./data/4th-t2/train.csv')
test=pd.read_csv('data/4th-t2/test.csv')

In [217]:
train.describe(include='O')

,Gender,Ever_Married,Graduated,Profession,Spending_Score,Var_1
count,6665,6665,6665,6665,6665,6665
unique,2,2,2,9,3,7
top,Male,Yes,Yes,Artist,Low,Cat_6
freq,3677,3944,4249,2192,3999,4476


In [ ]:
# 범주형 변수
# train.select_dtypes(include='object').columns
# ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score','Var_1']
cat_cols = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score','Var_1']

In [219]:
## label encoding
## Series.astype('category').cat.codes
train['Gender'] = train['Gender'].astype('category').cat.codes
train['Ever_Married'] = train['Ever_Married'].astype('category').cat.codes
train['Graduated'] = train['Graduated'].astype('category').cat.codes
train['Profession'] = train['Profession'].astype('category').cat.codes
train['Spending_Score'] = train['Spending_Score'].astype('category').cat.codes
train['Var_1'] = train['Var_1'].astype('category').cat.codes
train

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,1,0,22,0,5,1.0,2,4.0,3,4
1,466315,0,1,67,1,2,1.0,2,1.0,5,2
2,461735,1,1,67,1,7,0.0,1,2.0,5,2
3,461319,1,1,56,0,0,0.0,0,2.0,5,3
4,460156,1,0,32,1,5,1.0,2,3.0,5,3
...,...,...,...,...,...,...,...,...,...,...,...
6660,463002,1,1,41,1,0,0.0,1,5.0,5,2
6661,464685,1,0,35,0,4,3.0,2,4.0,3,4
6662,465406,0,0,33,1,5,1.0,2,1.0,5,4
6663,467299,0,0,27,1,5,1.0,2,4.0,5,2


In [221]:
## label encoding
test['Gender'] = test['Gender'].astype('category').cat.codes
test['Ever_Married'] = test['Ever_Married'].astype('category').cat.codes
test['Graduated'] = test['Graduated'].astype('category').cat.codes
test['Profession'] = test['Profession'].astype('category').cat.codes
test['Spending_Score'] = test['Spending_Score'].astype('category').cat.codes
test['Var_1'] = test['Var_1'].astype('category').cat.codes
test

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,0,1,36,1,2,0.0,2,1.0,5
1,458994,1,1,37,1,5,8.0,0,4.0,5
2,459000,1,1,59,0,4,11.0,1,2.0,5
3,459003,1,1,47,1,1,0.0,1,5.0,3
4,459005,1,1,61,1,1,5.0,2,3.0,5
...,...,...,...,...,...,...,...,...,...,...
2149,467950,0,0,35,1,3,1.0,2,2.0,5
2150,467954,1,0,29,0,5,9.0,2,4.0,5
2151,467958,0,0,35,1,1,1.0,2,1.0,5
2152,467961,1,1,47,1,4,1.0,1,5.0,3


In [223]:
# ID, target 처리
target = train.pop('Segmentation')
train = train.drop("ID", axis=1)
test_ID = test.pop('ID')

In [225]:
# 모델 선택
# 하이퍼파라미터 튜닝: max_depth, n_estimators
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0, max_depth=7, n_estimators=500)

In [227]:
# 교차 검증
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, train, target, scoring='f1_macro', cv=5)
print(scores)
print(scores.mean())

[0.53130191 0.51695963 0.52121909 0.54069647 0.51119827]
0.524275072755451


In [229]:
rf.fit(train, target)
pred = rf.predict(test)
pred

array([1, 3, 2, ..., 1, 2, 4], dtype=int64)

In [231]:
submit = pd.DataFrame({
    'ID': test_ID,
    'Segmentation': pred
})
submit.to_csv("submission.csv", index=False)
# Score: 0.32046

# XGBOOST, 라벨인코딩, 평가지표

In [280]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# 데이터 불러오기
train = pd.read_csv('./data/4th-t2/train.csv')
test = pd.read_csv('./data/4th-t2/test.csv')

In [282]:
# 타겟 분리
target = train.pop('Segmentation')
test_ID = test.pop('ID')
train_ID = train.pop('ID')

In [284]:
cat_cols=train.select_dtypes(include='object').columns
print(cat_cols)

Index(['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score',
       'Var_1'],
      dtype='object')


In [286]:
for col in cat_cols:
    le=LabelEncoder()
    le.fit(list(train[col])+list(test[col])) # train+ test 전체 기준으로 인코딩
    train[col]= le.transform(train[col])
    test[col]=le.transform(test[col])

In [288]:
# 결측치 처리 (간단히 평균으로)
train = train.fillna(train.mean(numeric_only=True))
test= test.fillna(test.mean(numeric_only=True))

In [295]:
y= target-1

In [299]:
X_train, X_val, y_train, y_val = train_test_split(train, y, test_size=0.2, random_state=42, stratify=target)

In [301]:
model = XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)
model.fit(X_train, y_train)

C:\Users\pjjeo\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:25:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [303]:
# 검증 점수
val_pred = model.predict(X_val)
f1 = f1_score(y_val, val_pred, average='macro')
print(f"Validation Macro F1-score: {f1:.4f}")

Validation Macro F1-score: 0.5046


In [309]:
# 전체 학습 후 test 예측
model.fit(train, y)
pred = model.predict(test)

C:\Users\pjjeo\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:26:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [311]:
final_pred= pred+1
# 제출 파일 저장
submit = pd.DataFrame({
    'ID': test_ID,
    'Segmentation': final_pred
})
submit.to_csv('submission.csv', index=False)

# 시험 맞춤형 최적화

In [360]:
train =pd.read_csv('./data/4th-t2/train.csv')
test=pd.read_csv('data/4th-t2/test.csv')

In [362]:
train.describe(include="O") # 범주형(object 타입) 변수들에 대한 요약 통계 

train= pd.get_dummies(train)
test=pd.get_dummies(test)

target = train.pop('Segmentation')
test_ID = test.pop('ID')
train_ID = train.pop('ID')

In [364]:
train.shape, test.shape

((6665, 28), (2154, 28))

In [366]:
# 모델 선택
# 하이퍼파라미터 튜닝: max_depth, n_estimators
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0, max_depth=8, n_estimators=800)

In [368]:
# 교차 검증
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, train, target, scoring='f1_macro', cv=5)
print(scores)
print(scores.mean())

[0.5245382  0.51940186 0.52203272 0.53952346 0.50456113]
0.5220114744092124


In [347]:
# 학습
rf.fit(train, target)
pred = rf.predict(test)
pred

array([1, 3, 2, ..., 1, 2, 4], dtype=int64)

In [326]:
submit = pd.DataFrame({
    'ID': test_ID,
    'Segmentation': pred
})
submit.to_csv("submission.csv", index=False)
# Score: 0.32046

| 유형          | 예                    | 비중           | 평가 지표                       |
| ----------- | -------------------- | ------------ | --------------------------- |
| ✅ **이진 분류** | 구매 여부, 이탈 여부         | **가장 자주 나옴** | `ROC-AUC`, `accuracy`, `f1` |
| ✅ **다중 분류** | 고객 등급 (1\~4), 사용자 유형 | **최근 증가 추세** | `macro F1-score` (지금 푼 문제)  |
| ⬛ **회귀**    | 판매량, 금액 예측           | **가끔 출제**    | `RMSE`, `MAE`               |


🧠 유형별 준비 포인트
✅ 1. 이진 분류

    LogisticRegression, RandomForestClassifier, XGBClassifier

    타겟: 0/1

    roc_auc_score() 많이 사용됨

✅ 2. 다중 분류

    RandomForestClassifier, XGBClassifier → multi:softmax or 기본

    평가: macro F1 중요 (클래스 비율 불균형 대비)

✅ 3. 회귀

    LinearRegression, RandomForestRegressor, XGBRegressor

    타겟: 연속형

    평가: mean_squared_error(..., squared=False) → RMSE